In [ ]:
import pandas

In [ ]:
!pip install datasets -q --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


# Q1

## Downloading datasets


In [ ]:
!rm -rf /content/datasets
!mkdir /content/datasets

### VoxCeleb1-H

In [ ]:
try:
  import datasets
except Exception as e:
  print(e)

In [ ]:
dataset = datasets.load_dataset("Codec-SUPERB/Voxceleb1_test_original")["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4874 [00:00<?, ? examples/s]

In [ ]:
!pip show fairseq

In [ ]:
import tqdm

index_to_filepath_mapping = {}
filepath_to_index_mapping = {}
for i in range(len(dataset)):
  sample = dataset[i]
  sample_id = sample['id']
  folder_id, yt_id, file_id = sample_id.split('+')
  filepath = sample_id.replace('+', '/')
  index_to_filepath_mapping[i] = filepath
  filepath_to_index_mapping[filepath] = i

assert len(filepath_to_index_mapping) == len(index_to_filepath_mapping), "error here!"
dataset.index_to_filepath = index_to_filepath_mapping
dataset.filepath_to_index = filepath_to_index_mapping

In [ ]:
!mkdir /content/datasets/voxceleb1h
!wget https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/list_test_hard.txt -O /content/datasets/voxceleb1h/list_test_hard.txt

--2024-04-02 14:35:25--  https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/list_test_hard.txt
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34257232 (33M) [text/plain]
Saving to: ‘/content/datasets/voxceleb1h/list_test_hard.txt’

/content/datasets/v 100%[===================>]  32.67M  7.69MB/s    in 4.2s    

2024-04-02 14:35:31 (7.69 MB/s) - ‘/content/datasets/voxceleb1h/list_test_hard.txt’ saved [34257232/34257232]



In [ ]:
with open('/content/datasets/voxceleb1h/list_test_hard.txt', 'r') as f:
  list_test_hard = [line.strip('\n') for line in f.readlines()]

In [ ]:
from torch.utils.data import Dataset, DataLoader
from datasets import Dataset, Audio
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

class VoxDatasetTest(Dataset):
  def __init__(self, dataset, list_test_hard):
    self.dataset = dataset
    self.audios = []
    self.padding_fn = []
    padding_value = 0.0
    self.maxlen = 0
    for i in tqdm(range(len(dataset))):
      sample = dataset[i]
      self.audios.append(sample)
      self.maxlen = max(self.maxlen, len(sample['audio']['array']))
      self.padding_fn.append(
          torch.nn.ConstantPad1d((0, self.maxlen - len(sample['audio']['array'])), padding_value)
      )
    self.list_test_hard = list_test_hard


    sampling_rate = 16000
    self.list_audio1 = []
    self.list_audio2 = []
    self.list_speaker_matches = []
    self.padding_fns = []
    for line in tqdm(self.list_test_hard):
      a, b, c = line.split()
      if b not in dataset.filepath_to_index or c not in dataset.filepath_to_index:
        continue

      a = int(a)
      d, e = self.padding_fn[dataset.filepath_to_index[b]], self.padding_fn[dataset.filepath_to_index[c]]
      b, c = self.audios[dataset.filepath_to_index[b]]['audio']['array'], self.audios[dataset.filepath_to_index[c]]['audio']['array']


      self.list_speaker_matches.append(a)
      self.list_audio1.append(b)
      self.list_audio2.append(c)
      self.padding_fns.append([d, e])


  def __len__(self):
    return len(self.list_audio1)

  def __getitem__(self, idx):
    if type(idx) == type(1):
      a, b, c = self.list_speaker_matches[idx], self.list_audio1[idx], self.list_audio2[idx]
      d, e = self.padding_fns[idx]
      print('bruh2')
      b = d(torch.Tensor(b))
      c = e(torch.Tensor(c))
      return a, b, c
    else:
      batch = []
      for id in idx:
        print('okay')
        batch.append(self.__getitem__(id))
      return batch

voxdataset = VoxDatasetTest(dataset, list_test_hard)
print('dataset loaded')
print(len(voxdataset))
sample = voxdataset[0]
print(sample)

100%|██████████| 552536/552536 [00:00<00:00, 1377280.21it/s]


dataset loaded
9389
bruh2
(1, tensor([0.0068, 0.0064, 0.0049,  ..., 0.0000, 0.0000, 0.0000]), tensor([0.0009, 0.0030, 0.0008,  ..., 0.0000, 0.0000, 0.0000]))


In [ ]:



batch_size = 16

def voxcollate(batch):
  print('inside collate')
  targets = torch.Tensor([sample[0] for sample in batch]).long()
  audio1s = torch.Tensor([sample[1] for sample in batch])
  audio2s = torch.Tensor([sample[2] for sample in batch])
  return targets, audio1s, audio2s



voxdataloader = DataLoader(voxdataset, batch_size = batch_size)

try:
  for targets, audio1s, audio2s in voxdataloader:
    print('something')
    break
  print(targets.shape)
  print(audio1s.shape)
  print(audio2s.shape)
except Exception as e:
  print(e)

okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2
okay
bruh2


TypeError: list indices must be integers or slices, not tuple

In [ ]:
import torch
import numpy as np
from types import SimpleNamespace
from transformers import AutoConfig, AutoModel



wav2vec2_xlsr = SimpleNamespace(**{})
wav2vec2_xlsr.model_tag = 'facebook/wav2vec2-large-xlsr-53'
wav2vec2_xlsr.autoconfig = AutoConfig.from_pretrained(wav2vec2_xlsr.model_tag)
wav2vec2_xlsr.automodel = AutoModel.from_config(wav2vec2_xlsr.autoconfig)
wav2vec2_xlsr.automodel.eval()
print()



In [ ]:
print(list(dataset.filepath_to_index.keys())[0])

id10270/5r0dWxy17C8/00001.wav


### Kathbath

In [ ]:
!wget -c --tries=0 --read-timeout=20 https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip -P datasets/librispeech/LibriMix/storage_dir

In [ ]:
!mv datasets/librispeech/LibriMix/storage_dir/wham_noise.zip /content/wham_noise.zip

In [ ]:
!unzip -l /content/wham_noise.zip

In [ ]:
!unzip wham_noise.zip "wham_noise/tt/*" -d /content/

In [ ]:
!du -sh /content/wham_noise

In [ ]:
# !zip -r /content/tt.zip /content/wham_noise/tt

In [ ]:
!pip install datasets

In [ ]:




#####################################################


In [ ]:
import librosa

In [ ]:
from IPython import display
import os
from tqdm import tqdm

tt_dir  = '/content/wham_noise/tt'

max_duration = 0
for path in tqdm(os.listdir(tt_dir)):
  path = os.path.join(tt_dir, path)
  duration = librosa.get_duration(filename=path)
  max_duration = max(max_duration, duration)

sample_filename = os.path.join(tt_dir, os.listdir(tt_dir)[0])
print(f'max duration in test-clean:  {max_duration}')
display.Audio(sample_filename)

In [ ]:

from datasets import Dataset, Audio
import math

def load_audio_dataset(list_paths):
  audio_dataset = Dataset.from_dict(
    {
        "audio": list_paths
    }
  ).cast_column("audio", Audio(sampling_rate=16000))
  return audio_dataset


import os
tt_dir= '/content/wham_noise/tt'
list_audio_paths = [os.path.join(tt_dir, path) for path in os.listdir(tt_dir) if path.endswith('.wav')][:2]
audio_dataset = load_audio_dataset(list_audio_paths)

print(audio_dataset[0]['audio']['array'].shape, audio_dataset[1]['audio']['array'].shape)

padding_value = 0.0
# max_len = max([len(audio_dataset[i]['audio']['array']) for i in range(len(audio_dataset))])
max_len = 16000 * math.ceil(max_duration)

padding_fn = [torch.nn.ConstantPad1d(
    (0, max_len - len(audio_dataset[i]['audio']['array'])), padding_value) for i in range(len(audio_dataset))]
audio_batch = torch.cat([
    padding_fn[i](
        torch.Tensor(audio_dataset[i]['audio']['array']).reshape(1, -1)
    )
    for i in range(len(audio_dataset))])

batch_output = wav2vec2_xlsr.automodel.feature_extractor(audio_batch)
batch_output.shape

In [ ]:
wav2vec2_xlsr.autoconfig

In [ ]:
type(audio_dataset[0]['audio']['array'])

In [ ]:
sample = torch.Tensor(audio_dataset[0]['audio']['array']).reshape(1, -1)
test_output = wav2vec2_xlsr.automodel.feature_extractor(sample)

In [ ]:
for i in range(len(audio_dataset)):
  print(
      np.mean(np.abs(audio_dataset[i]['audio']['array'])) \
      / np.mean(audio_dataset[i]['audio']['array'])
  )
  print()

In [ ]:
type(test_output)

In [ ]:
test_output.shape

In [ ]:
wav2vec2_xlsr.autoconfig

In [ ]:
wav2vec2_xlsr.automodel.feature_extractor.forward

In [ ]:
wav2vec2_xlsr.automodel

In [ ]:
!zip /content/tt.zip --out /content/new_tt.zip -s 100m

In [ ]:
files.download('new_tt.zip')

In [ ]:
for i in range(1, 20):
  numeral = f'0{i}'[-2:]
  path = f'new_tt.z{numeral}'
  if os.path.exists(path):
    files.download(path)
  else:
    print(path)

In [ ]:
from google.colab import files
files.download("/content/tt.zip")

In [ ]:
!git clone https://github.com/JorisCos/LibriMix.git

In [ ]:
%cd LibriMix

In [ ]:
!pip install -r requirements.txt

In [ ]:
# !cat README.md
# import os
!bash generate_librimix.sh storage_dir